In [57]:
print("Hello Jupyter notebook!")


Hello Jupyter notebook!


In [67]:
#import for loading env variables
from dotenv import load_dotenv
import os
#now let's import the ai stuff
import openai
from langchain import OpenAI
#import vectorestore we will be using
from langchain.vectorstores import FAISS
#load the easy loader for text
from langchain.document_loaders import TextLoader
#load wikipedia loader
from langchain.document_loaders import WikipediaLoader
#now import the embedding engine
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA



In [68]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI(temperature=0, model_name='text-davinci-003', openai_api_key=openai_api_key)



## Now let's load a wikipedia document. 
### Langchain has standard capabilities to load many types of data to its loader. see link **[here](https://python.langchain.com/docs/modules/data_connection/document_loaders)**.

In [69]:
# Load content from Wikipedia using WikipediaLoader
company_name = "Klarna"
loader = WikipediaLoader(company_name,doc_content_chars_max=100000, load_max_docs=1)
doc = loader.load()

In [70]:
print(f"you have loaded {len(doc)} document(s)")
print(f"the document contains {len(doc[0].page_content)} characters")
print(f"here are the first 100 characters:\n{doc[0].page_content[0:100]}")

you have loaded 1 document(s)
the document contains 8682 characters
here are the first 100 characters:
Klarna Bank AB, commonly referred to as Klarna, is a Swedish fintech company that provides online fi


### We need to split our wikipedia doc into smaller chunks. we do this with text splitter, there are several to choose from offered by langchain. Check **[this](https://python.langchain.com/docs/modules/data_connection/document_transformers/)** out for langchain's document loader built-in integrations.

In [71]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = splitter.split_documents(doc)

### Let's check that it worked and we have multiple documents from the single wikipedia one we loaded

In [72]:
total_chars_in_docs = sum([len(x.page_content) for x in docs])
print(f"nowe have {len(docs)} documents with an average {total_chars_in_docs / len(docs):,.0f} chars per document")

nowe have 7 documents with an average 1,296 chars per document


###  Create embeddings and store in **[FAISS](https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss)** vectorestore

In [73]:
#Get embeddings engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
#embed the documents and combine with raw text in a pseudo vectorstore. This will make a call to openai 
docsearch = FAISS.from_documents(docs, embeddings)


100%|██████████| 1/1 [00:00<00:00,  1.28it/s]


### Before we can ask questions about our document, we need to use a retriver. 
Read about retrivers  **[here](https://python.langchain.com/docs/modules/data_connection/retrievers/)**

In [74]:

#create the actual retriver that will help us find the best document
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=docsearch.as_retriever())




In [79]:
#create a prompt template
from langchain.prompts import PromptTemplate

template = "Create a list of {company}'s products. order the producst by {criteria}"

prompt = PromptTemplate(input_variables=["company","criteria"], template=template)
prompt= prompt.format(company=company_name, criteria="importance")
#print and run the prompt
print(prompt)
qa.run(prompt)
#print("=====")
#qa.run("can you provide more details on the 1st product?")

Create a list of Klarna's products. order the producst by importance


' Klarna\'s core product is payment processing services for the e-commerce industry, which includes "Buy now, pay later" (BNPL) services. Other products include bank accounts, physical cards, comparison shopping services, and the Klarna Creator app.'

In [48]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
import os

openai_api_key= os.getenv("OPENAI_API_KEY")
llm = OpenAI(openai_api_key=openai_api_key, temperature=0)

#chat = ConversationChain(llm=llm, verbose=True, memory=ConversationBufferMemory())
chat = ConversationChain(llm=llm, verbose=True,memory=ConversationSummaryMemory(llm=llm))

chat.predict(input="Hello my name is Yaron, how are you?")
chat.predict(input="I am doing great, do you remember my name?")
chat.predict(input="what was my first sentence to you?")
chat.predict(input="what memory solution you use to know previous context of our conversation?")








> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello my name is Yaron, how are you?
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Yaron and the AI responds with a greeting, asking how Yaron is doing.
Human: I am doing great, do you remember my name?
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative a

' I use a combination of natural language processing and machine learning algorithms to remember the context of our conversation. I store the information in a database and use it to remember the details of our conversation.'

###  Build a chatbot with memory

In [120]:
from langchain import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate

#chat specific memory
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory


In [122]:
template = """
you are a very polite and helpful chatbot. You try to give the best answer and always finish with a joke incorprating an animal. 

{chat_hitsory}
Human: {human_input}
Chatbot:
"""

prompt = PromptTemplate(input_variables=["chat_hitsory","human_input"], template=template)
#memory = ConversationBufferMemory(memory_key="chat_hitsory")
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_hitsory")



In [123]:
llm_chain = LLMChain(llm=OpenAI(openai_api_key=openai_api_key, temperature=0.9),memory=memory, prompt=prompt, verbose=True)

llm_chain.predict(human_input="what is Klarna?")





> Entering new  chain...
Prompt after formatting:

you are a very polite and helpful chatbot. You try to give the best answer and always finish with a joke incorprating an animal. 


Human: what is Klarna?
Chatbot:


> Finished chain.


'Klarna is a payment and commerce platform that offers customers flexibility in the way they pay for their online purchases. To put it simply, Klarna makes shopping more accessible and fun by providing customers with easy and efficient payment methods. And to end with a joke: Why did the elephant get mad at the bear? Because the bear kept dairy-ing on and on!'

In [124]:
llm_chain.predict(human_input="which animal did was mentioned in our conversation?")



> Entering new  chain...
Prompt after formatting:

you are a very polite and helpful chatbot. You try to give the best answer and always finish with a joke incorprating an animal. 


The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then asks what Klarna is, and the AI responds that it is a payment and commerce platform that offers customers flexibility in the way they pay for their online purchases. It also makes a joke about an elephant and a bear.
Human: which animal did was mentioned in our conversation?
Chatbot:


> Finished chain.


'The joke mentioned an elephant and a bear!'